# For thermomajorization with thermal states having rational entries.

**Another approch as explained in appendix B**

The following set of finite conditions are sufficient to transform a state $\rho$ with the vector of eigenvalues denoted by $q_\rho$ and is block diagonal in the energy eigenbasis with arbitrary accuracy into another block diagonal state $\sigma$ with the vector of eigenvalues denoted by $q_\sigma$ using catalytic thermal operations, if following conditions hold.

Choose
$r=\frac{\log N}{\log x_1- \log \lbrace y_1\left( 1+\frac{ϵ}{g_{min}}\right)^{2} \rbrace}$  and   $s=\frac{\log N}{\log y_{min} - \log \lbrace x_{min}\left( 1+\frac{ϵ}{g_{min}}\right)^{2} \rbrace}$

1. For $r>0$ and $\bar r=\lfloor r+1\rfloor$,
\begin{align}
F_{k,\bar r}(\mathbf x)< F_{k,\bar r}(\mathbf y) G(k,r), \qquad \bar r\leq k\leq n\bar r,
\end{align}
Here $G(k,r)$should be such that $\max\limits_k G(k,r) \leq \frac{1}{A(\epsilon)}$ with \
$$\frac{1}{A_1(\epsilon)}\leq 2^{ -\frac{1}{N}  {\left(1+ \frac{\epsilon}{g_{\min}} \right)}^{2(r+1)} }~~~\text{for}~~ p\in (1,r+1)$$
and
$$\frac{1}{A_2(\epsilon)}\leq 2^{ -\frac{2\epsilon}{N g_{\min}} }~~~\text{for} ~~p\in (0,1).$$
Above conditions implies,
$$\max\limits_k G(k,r) \leq \min \left\{ \frac{1}{A_1(\epsilon)},\frac{1}{A_2(\epsilon)}\right\}$$

2. $H_1(\mathbf x)< H_1(\mathbf y) - 2 \log\left(1+ \frac{\epsilon}{g_{\min}} \right)$,\
and either \\
3.
 (a) the weight of $\mathbf y$ is smaller than $n$, \
 or,\
 (b) the weight of $\mathbf y$ is $n$, $s>0$ with
 \begin{align}
 s=\frac{\log n}{\log x_{\min}-\log y_{\min}} \nonumber
 \end{align}
and
\begin{align}
F_{k,1}\left(\frac{1}{\mathbf x^{\bar s}}\right) /A_3(ϵ)> F_{k,1}\left(\frac{1}{\mathbf y^{\bar s}}\right), \qquad 1\leq k\leq n \ \nonumber.
\end{align}
with $\frac{1}{A_3(\epsilon)}\leq 2^{ -\frac{\left(1+ \frac{\epsilon}{g_{\min}} \right)^{2(1+s)}-1}{N }}.$



In [ ]:
# for checking the trumping conditions for thermal states

from scipy.optimize import minimize_scalar
import math
from itertools import product

# For rearranging the list in descending order
def descending_order(list_vector):
    list_vector.sort(reverse=True)
    return list_vector

# maximum number of elements of a vector
def compute_n(vector_a, vector_b):
    return max(len(vector_a), len(vector_b))

# for defining thermal state uniformly distributed accress its entries
def g_uniform(length_x):
    n = len(length_x)
    each_entry = 1.0 / n  # Since the list is uniform, each value should be 1/n

    # Generate the uniform list
    uniform_list = [each_entry] * n
    return uniform_list

# The symmetric polynomial
def F(k, r, x):
    n = compute_n(x, y)  # Number of elements in the list
    result = 0.0

    # Generate all possible combinations of (k1, k2, ..., kn) with 1 <= ki <= r
    for k_comb in product(range(1, r), repeat=n):

        if sum(k_comb) == k:
            product_term = math.prod([x[i]**k_comb[i] / math.factorial(k_comb[i]) for i in range(n)])
            result += product_term
    return result

#compute r(x, y) and s(x, y)
def compute_r(x, y, eps, g_min):
    return math.log(compute_n(x, y)) / (math.log(descending_order(x)[0]) -\
           math.log(descending_order(y)[0]*(1. + eps/g_min)**2 ))

def compute_s(x, y, eps, g_min):
    return math.log(compute_n(x, y)) / (math.log(min(y)) - math.log(min(x)*(1. + eps/g_min)**2 ))

# Calculate the p-norm of a vector
def p_norm(vector, p):
    norm_sum = sum(abs(x) ** p for x in vector)
    norm = norm_sum ** (1 / p)
    return norm

def reverse_A1(n,eps,g_min,r):
    return 2**(-(1./n) * ((1. + eps/g_min)**(2*(r+1.))))

def reverse_A2(n,eps,g_min):
    return 2**(-2* eps/(n * g_min))

def reverse_A3(n,eps,g_min,s):
    return 2**(-((1. + eps/g_min)**(2*(s+1.))-1.)/n)

def find_G(n, x, y, eps, g_min):
    # Minimize reverse_A1 in the range p = 1 to r
    r_new = math.floor(compute_r(x, y, eps, g_min))
    s_new = math.floor(compute_s(x, y, eps, g_min))
    min_A1 = reverse_A1(n,eps,g_min,r_new)
    # Minimize reverse_A2 in the range p = 0 to 1
    min_A2 = reverse_A2(n,eps,g_min)
    # Minimize reverse_A3 in the range p = -s, to 0
    #min_A3 = reverse_A3(n,eps,g_min,s_new)
    # Return the overall minimum
    return min(min_A1, min_A2)

def check_condition_thermo_irr_1(x, y, eps, g_min):
    r_new = math.floor(compute_r(x, y, eps, g_min) + 1)
    n = compute_n(x, y)
    k_max = n* r_new

    for k in range(r_new, k_max + 1):
        Fx = F(k, r_new, x)
        Fy = F(k, r_new, y)
        if Fx > find_G(n, x, y, eps, g_min)* Fy:
            return "Trumping not possible"
    return "Condition 1 satisfied"

# For checking the second condition
def H1(x):
    result = 0
    for xi in x:
        if xi > 0:  # Avoid log(0) which is undefined
            result += -xi * math.log(xi)
    return result

def check_condition_thermo_irr_2(x, y, eps, g_min):
    if H1(x) < H1(y) - 2*math.log(1. +eps/g_min):
        return "Trumping not possible"
    return "condition 2 satisfied"


# For checking the third condition

def reciprocal_vector(x, s):
    return [1 / xi**s if xi != 0 else float('inf') for xi in x]

#def find_G1(n, x, y, eps, g_min):

def check_condition_thermo_irr_3(x, y, eps, g_min):
    s = max(0, math.floor(compute_s(x, y, eps, g_min)+1))
    n = compute_n(x, y)

    for k in range(1, len(x) + 1):
        F_x = F(k, 1, reciprocal_vector(x, s))
        F_y = F(k, 1, reciprocal_vector(y, s))
        if F_x* reverse_A3(n, eps, g_min, s) < F_y:
            return "Trumping not possible"

    return "condition 3 satisfied"

#y = [0.6098126600657403, 0.3046464157597675, 0.043520852518815774, 0.04202007165567632]
#x = [0.7313456157625823, 0.12116969758203461, 0.13747896166533685, 0.010005724990046306]
y = [0.862942,0.129846,0.00558697,0.00162474]
x = [0.936918,0.0467542,0.0159775,0.000350242]
g = [0.71, 0.21, 0.06, 0.02]
g_uniform = g_uniform(g)
differences = [abs(a - b) for a, b in zip(g, g_uniform)]
eps = min(differences)
g_min = min(g)
result = check_condition_thermo_irr_1(x, y, eps, g_min)
print("Result:", result)

result = check_condition_thermo_irr_2(x, y, eps, g_min)
print("Result:", result)

result = check_condition_thermo_irr_3(x, y, eps, g_min)
print("Result:", result)